# Image analysis with Python
This notebook provides an image analysis for various kinds of cultural heritage / museum images.
It is based on the following tutorials and discussions:

**[1]** https://towardsdatascience.com/building-an-image-color-analyzer-using-python-12de6b0acf74 \
**[2]** https://stackoverflow.com/questions/13613573/how-to-speed-up-color-clustering-in-opencv \
**[3]** https://medium.com/codex/rgb-to-color-names-in-python-the-robust-way-ec4a9d97a01f \
**[4]** https://stackoverflow.com/questions/30230592/loading-all-images-using-imread-from-a-given-folder 

This notebook provides analyses for the CH images we got from Coding da Vinci. It preparats the images for our IR feature as well as it gives detailed explainations about the concepts behind. See at the end of the notebook for answers to questions regarding the concepts if you wonder about our idea.


In [ ]:
!pip install extcolors

In [ ]:
!pip install webcolors

In [ ]:
import extcolors

# plot color distribution
import matplotlib.pyplot as plt
import numpy as np
# path 


# other
from tqdm import tqdm
import random

#### Data
We work with OpenGLAM data. This maens we use data that is free for reuse licences by the institutions that give it. Our following image data is especially taken from the context of Coding da Vinci. All data sets were published there and licensed freely for reuse with the purpose to engage tech and culture communities to work with the data. 
We will get our data per URL. [6]

Please load the following folders into the image subdirectory of this repository:
- folder name "westmuensterland": https://download.codingdavinci.de/index.php/s/y7wHa8r6dWtnTTm?path=%2F -- This is the most homogenous of our datasets, containing XXX images of lamps from the XXX museum, which are fotografed in the sammer manner. [Bild]
- folder name "saarlandmuseum": https://download.codingdavinci.de/index.php/s/3y5dXdFAmGxAGgzhttps://download.codingdavinci.de/index.php/s/3y5dXdFAmGxAGgz
- folder name "muenchen":

Since Cultural Heritage images are mostly provided in high resolution for reusing purposes, we directly preprocess them when loading them from the folder and save only the preprocessed (and therefore downsized) images.

#### Color analysis
After preprocessing we perform the color analysis. Since we will later map the colors values to their closest colour name in CSS3 style we use RGB values for that.
[1] provides us with a plot of the color analysis, which we will use for demonstration purposes but what will be not necessary for the future large-scale analysis.
The code originally belongs to [4].

In [ ]:
def rgb_to_hex(rgb_color):
    hex_color = "#"
    for i in rgb_color:
        i = int(i)
        hex_color += ("{:02x}".format(i))
    return hex_color

In [ ]:
from scipy.spatial import KDTree
from webcolors import (
    CSS3_HEX_TO_NAMES,
    hex_to_rgb)

def convert_rgb_to_names(rgb_tuple):
    # a dictionary of all the hex and their respective names in css3
    css3_db = CSS3_HEX_TO_NAMES
    names = []
    rgb_values = []
    for color_hex, color_name in css3_db.items():
        names.append(color_name)
        rgb_values.append(hex_to_rgb(color_hex))

    kdt_db = KDTree(rgb_values)
    distance, index = kdt_db.query(rgb_tuple)
    return names[index]

For color analysis we used two different approaches:
- k-means: which divides the image in five clusters and gives back the dominant color of each cluster
- `extcolors`: which is a packe providing the `extract_from_image`-function that gives back the RGB-value for each pixel of the file
As one can see for the example below k-means needs prepared (reshaped) images and is much slower on the original file size. Since we will work with many high-resolution images we will use 'extcolors`. 
Both algorithms are much faster on resized images (e.g. down-scaling to 50 percent of the original size).

*Example*

In [ ]:
import extcolors
def color_analysis_ext(image_path, file_path):
    colors, pixel_count = extcolors.extract_from_path(image_path)
    colors = dict(colors)
    names = {convert_rgb_to_names(key): value/sum(colors.values()) for (key, value) in colors.items()}
    file = open(file_path, "w+")
    line = str([name for name in names.keys()])
    file.write(line)
    return names     


In [ ]:
result = color_analysis_ext("C:\\Users\\LeaGl\\Documents\\GitHub\\InformationRetrieval\\images\\westmuensterland\\HM-17-44.jpg","C:\\Users\\LeaGl\\Documents\\GitHub\\InformationRetrieval\\images\\westmuensterlandtext\\HM-17-44.txt")

plt.xticks(rotation='vertical')
plt.plot(list(result.keys()), list(result.values()))
plt.show()

plt.xticks(rotation='vertical')
plt.plot(list(result.keys()), list(result.values()))
plt.yscale('log')
plt.show()

In [ ]:
import PIL 
img = PIL.Image.open("C:\\Users\\LeaGl\\Documents\\GitHub\\InformationRetrieval\\images\\westmuensterland\\HM-17-44.jpg")
width, height = img.size
new_size = (int(width/2), int(height/2))
img = img.resize(new_size)
result = color_analysis_ext(img, "C:\\Users\\LeaGl\\Documents\\GitHub\\InformationRetrieval\\images\\westmuensterlandtext\\HM-17-44.txt")

plt.xticks(rotation='vertical')
plt.plot(list(result.keys()), list(result.values()))
plt.show()
#plt.savefig(name)

plt.xticks(rotation='vertical')
plt.plot(list(result.keys()), list(result.values()))
plt.yscale('log')
plt.show()

So one can see that in one image there is a power law on which the distribution of the colors depend. This varies with the downsizing of the image, which is the resaon why we will try to use the original sized images in the following and why we dropped the k-means approach.
We will now expand the data.

#### Full Analysis

In [ ]:
import os
def analyse_images(folder, file_folder):
    pbar = tqdm(total=len(os.listdir(folder)))
    color_dict_all = dict.fromkeys([el for el in CSS3_HEX_TO_NAMES.values()], 0)
    for filename in os.listdir(folder):
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff')): continue
        result = color_analysis_ext(os.path.join(folder, filename), os.path.join(file_folder) + "\\" + filename.split('.')[0] + ".txt") 
        for (key,value) in color_dict_all.items():
            if key not in result.keys(): continue
            color_dict_all[key] = value+result[key]
        pbar.update(n=1)
        result = {k: v for k, v in sorted(color_dict_all.items(), key=lambda item: item[1], reverse = True)}
        result = {k: v for k, v in result.items() if v >= 1} # delete colors with 0 percent, since log will not work on log(0). delete also 
        # colors that appear less then 1%
    return result

In [ ]:
result = analyse_images("C:\\Users\\LeaGl\\Documents\\GitHub\\InformationRetrieval\\images\\westmuensterland", "C:\\Users\\LeaGl\Documents\\GitHub\\InformationRetrieval\\images\\westmuensterlandtext")

plt.xticks(rotation='vertical')
plt.plot(list(result.keys()),list(result.values()))
plt.show()
plt.savefig("westmuensterland")

plt.xticks(rotation='vertical')
plt.plot(list(result.keys()), list(result.values()))
plt.yscale('log')
plt.show()
plt.savefig("westmuensterland_log")

In [ ]:
result = analyse_images("C:\\Users\\LeaGl\\Documents\\GitHub\\InformationRetrieval\\images\\saarlandmuseum", "C:\\Users\\LeaGl\\Documents\\GitHub\\InformationRetrieval\\images\\saarlandmuseumtext")

plt.xticks(rotation='vertical')
plt.plot(list(result.keys()), list(result.values()))
plt.show()
plt.savefig("saarlandmuseum")

plt.xticks(rotation='vertical')
plt.plot(list(result.keys()), list(result.values()))
plt.yscale('log')
plt.show()
plt.savefig("saarlandmuseum_log")

In [ ]:
result = analyse_images("C:\\Users\\LeaGl\\Documents\\GitHub\\InformationRetrieval\\images\\muenchen", "C:\\Users\\LeaGl\Documents\\GitHub\\InformationRetrieval\\images\\muenchentext")

plt.xticks(rotation='vertical')
plt.plot(list(result.keys()),list(result.values()))
plt.show()
plt.savefig("muenchen")

plt.xticks(rotation='vertical')
plt.plot(list(result.keys()), list(result.values()))
plt.yscale('log')
plt.show()
plt.savefig("muenchen_log")